In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from math import *

In [3]:
try :
  with open('/kaggle/working/input.txt', 'r', encoding='utf-8') as f:
      text = f.read()
except:
  !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
  with open('input.txt', 'r', encoding='utf-8') as f:
      text = f.read()
print(len(text))

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.1*len(data)) # first 90% will be train, rest val
train_data = data[n:]
val_data = data[:n]

--2024-06-11 12:50:49--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  5.14MB/s    in 0.2s    

2024-06-11 12:50:50 (5.14 MB/s) - 'input.txt' saved [1115394/1115394]

1115394


https://github.com/r-gould/htm/tree/main

https://github.com/joedlcolvin/HTM-NLP

=> https://github.com/styalai/htm/

In [8]:
!pip install git+https://github.com/styalai/pyHTM

  Cloning https://github.com/styalai/pyHTM to /tmp/pip-req-build-5_9md_fl
  Running command git clone --filter=blob:none --quiet https://github.com/styalai/pyHTM /tmp/pip-req-build-5_9md_fl
  Resolved https://github.com/styalai/pyHTM to commit a1edf92f0a1ac42a2cce0100fdd92896f6938425
  Preparing metadata (setup.py) ... done
  Created wheel for PyHTM: filename=PyHTM-0.1-py3-none-any.whl size=20774 sha256=8f599b6b165d5e9546fe63201ea2b48bd3fa8b7075fd3fe02238793ef791d98d
  Stored in directory: /tmp/pip-ephem-wheel-cache-f7qet0la/wheels/7d/d9/0e/d160669be1f9b0d84dcc802e2c016e91c4bddbd6fee3a8e9c8
Successfully built PyHTM


In [4]:
!git clone https://github.com/styalai/htm/

Cloning into 'htm'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 34 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), 13.74 KiB | 265.00 KiB/s, done.


In [9]:
from pyhtm.pyhtm import ScalarEncoder, Regressor

In [10]:
from htm.components.layer import *

In [5]:
!python /kaggle/working/htm/main.py

tokenizer_config.json: 100%|███████████████████| 396/396 [00:00<00:00, 2.54MB/s]
tokenizer.json: 100%|██████████████████████| 2.11M/2.11M [00:00<00:00, 2.46MB/s]
special_tokens_map.json: 100%|████████████████| 99.0/99.0 [00:00<00:00, 656kB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|                                                 | 0/10000 [00:00<?, ?it/s](4096,)
[False False False ... False False False]
(4096,)
[False False False ... False False False]
(4096,)
[False False False ... False False False]
(4096,)
[False False False ... False False False]
^C
  0%|                                                 | 0/10000 [00:10<?, ?it/s]
Traceback (most recent call last):
  File "/kaggle/working/htm/main.py", line 119, in <module>
    layer = train(tokenizer, sdrs, layer_params, spatial_pooling_params, temporal_memory_params)
  File "/kaggle/working/htm/main.py", line 25, in train
    active_minicolumns = la

## Main

In [11]:
encoder = ScalarEncoder(n=50,w=15,minval=0,maxval=vocab_size)

In [12]:
encoder.encode(4).shape

(50,)

In [13]:
sdrlist = []
valuelist = []
for i in range(vocab_size):
    valuelist.append(i)
    sdrlist.append(encoder.encode(i))

reg = Regressor(sdrlist, valuelist, regressor_type = 'linear')
round(reg.translate(encoder.encode(23)))

R2 score on training SDRs: 0.9993662576039706


22

In [14]:
input_size = 50
layer_params = {
    "input_size": input_size, 
    "num_minicolumns": input_size, 
    "num_neurons_per_minicolumn": 32, 
    "num_proximal_synapses": input_size//2,
    "num_distal_segments": 0, 
    "max_distal_segments": 256,
    "max_distal_synapses": 256, 
    "proximal_overlap_thresh": 0,
    "proximal_thresh_perm": 0.1, 
    "distal_init_perm": 0.21, 
    "distal_thresh_perm": 0.5,
}

spatial_pooling_params = {
    "sparsity": 0.02,
    "delta_pos": 0.05, 
    "delta_neg": 0.008, 
    "boosting": False,
}

temporal_memory_params = {
    "sample_size": 32,
    "delta_pos": 0.1, 
    "delta_neg": 0.1, 
    "active_thresh": 16, 
    "match_thresh": 10,
}

In [15]:
from htm.components.layer import Minicolumn

class Layer:

    def __init__(self, input_size: int, num_minicolumns: int, num_neurons_per_minicolumn: int, num_proximal_synapses: int,
                 num_distal_segments: int, max_distal_segments: int, max_distal_synapses: int, proximal_overlap_thresh: int, 
                 proximal_thresh_perm: float, distal_init_perm: float, distal_thresh_perm: float):

        self.minicolumns = [Minicolumn(input_size, num_neurons_per_minicolumn, num_proximal_synapses, num_distal_segments, 
                                       max_distal_segments, max_distal_synapses, proximal_overlap_thresh, 
                                       proximal_thresh_perm, distal_init_perm, distal_thresh_perm)
                            for _ in range(num_minicolumns)]

        self.active_neurons = []
        self.winner_neurons = []
        self.active_segments = []
        self.matching_segments = []

    def run_spatial_pooling(self, x: np.ndarray[np.bool_], sparsity: float, learning_mode: bool,
                            delta_pos: float, delta_neg: float, boosting: bool) -> List[Minicolumn]:
        
        overlapped = []
        for minicolumn in self.minicolumns:
            is_active, score = minicolumn.proximal_segment.forward(x)
            if is_active:
                overlapped.append((minicolumn, score))

        overlapped.sort(key=lambda pair: pair[1], reverse=True)
        take_top = int(sparsity * len(self.minicolumns))
        active_minicolumns = [pair[0] for pair in overlapped[:take_top]]

        if learning_mode:
            for minicolumn in active_minicolumns:
                for synapse in minicolumn.proximal_segment.synapses:
                    synapse.hebbian_update(delta_pos, delta_neg, x)

                    if boosting:
                        raise NotImplementedError()
                        # increase boost coeff for cols that have low active duty   
                        # across all columns

        return active_minicolumns


    def run_temporal_memory(self, active_minicolumns: List[Minicolumn], sample_size: int, learning_mode: bool, 
                            delta_pos: float, delta_neg: float, active_thresh: int, match_thresh: int,
                            return_prediction: bool=False) -> Optional[List[Minicolumn]]:
        
        new_active_neurons = []
        new_winner_neurons = []
        new_active_segments = []
        new_matching_segments = []

        for minicolumn in self.minicolumns:
            if minicolumn in active_minicolumns:
                # minicolumn active
                active_distal_segments = set(minicolumn.distal_segments) & set(self.active_segments)
                if len(active_distal_segments) > 0:
                    # was correctly in predictive mode

                    for segment in active_distal_segments:
                        # activate neurons in predictive state
                        new_active_neurons.append(segment.origin_neuron)
                        new_winner_neurons.append(segment.origin_neuron)

                        # segments active => strengthen
                        # segments inactive => weaken
                        # grow synapses to previous winner neurons to strengthen pattern matching
                        if learning_mode:
                            for synapse in segment.receptive_synapses():
                                synapse.hebbian_update(delta_pos, delta_neg, self.active_neurons)

                            segment.grow_synapses(self.winner_neurons, sample_size)
                
                else:
                    # was not in predictive mode => bursting

                    # activate all neurons
                    for neuron in minicolumn.neurons:
                        new_active_neurons.append(neuron)

                    matching_distal_segments = set(minicolumn.distal_segments) & set(self.matching_segments)
                    if len(matching_distal_segments) > 0:
                        # if at least one matching segment,
                        # set the learning segment to the maximally matching segment
                        learning_segment = max(matching_distal_segments, key=lambda segment: (segment.num_active_synapses, random.random()))
                        winner_neuron = learning_segment.origin_neuron

                    else:
                        # if no matching segments,
                        # set the learning segment to a newly grown segment on the neuron with the fewest segments
                        winner_neuron = min(minicolumn.neurons, key=lambda neuron: (len(neuron.distal_segments), random.random()))
                        learning_segment = winner_neuron.grow_segment()

                    new_winner_neurons.append(winner_neuron)

                    if learning_mode:
                        for synapse in learning_segment.receptive_synapses():
                            synapse.hebbian_update(delta_pos, delta_neg, self.active_neurons)

                        learning_segment.grow_synapses(self.winner_neurons, sample_size)


            else:
                # minicolumn inactive
                if not learning_mode:
                    continue

                # this punishes (since minicol did not actually 
                # activate) segments that matched (which includes active segments)
                
                matching_distal_segments = set(minicolumn.distal_segments) & set(self.matching_segments)
                
                for segment in matching_distal_segments:
                    for synapse in segment.synapses:
                        if synapse.is_active(self.active_neurons):
                            synapse.perm = max([synapse.perm - delta_neg, 0.0])

        # update segment info using new info
        for minicolumn in self.minicolumns:
            minicolumn.update_distal_info()
            for segment in minicolumn.distal_segments:
                segment.update_active_count(new_active_neurons)

                # match_thresh < active_thresh, so since
                # segment.num_active_synapses > segment.num_active_receptive_synapses
                # then if a segment is active, it also matches.
                
                if segment.num_active_synapses >= match_thresh:
                    new_matching_segments.append(segment)

                if segment.num_active_receptive_synapses >= active_thresh:
                    new_active_segments.append(segment)

        
        # save new info
        self.active_neurons = new_active_neurons
        self.winner_neurons = new_winner_neurons
        self.active_segments = new_active_segments
        self.matching_segments = new_matching_segments

        if return_prediction:
            predicted_minicolumns: List[Minicolumn] = []
            for minicolumn in self.minicolumns:
                active_distal_segments = set(minicolumn.distal_segments) & set(self.active_segments)
                if len(active_distal_segments) > 0:
                    # minicolumn has been predicted to fire next
                    predicted_minicolumns.append(1)
                else:
                    predicted_minicolumns.append(0)
            return predicted_minicolumns

In [16]:
layer = Layer(**layer_params)

In [26]:
print(decode(train_data[:10].tolist()))
for i in tqdm(range(200)):
    for l in range(5):
        n = train_data[l]
        l = np.array(encoder.encode(n), dtype=bool)
        
        active = layer.run_spatial_pooling(l, **spatial_pooling_params, learning_mode=True)
        layer.run_temporal_memory(active, **temporal_memory_params, learning_mode=True)
        try:
            print(active.index(1))
        except:
            pass

        #print(decode( [round(reg.translate(predicted))] ), end="")

et, Marciu


  0%|          | 0/200 [00:00<?, ?it/s]

In [22]:
 for i in tqdm(range(100)):
    for l in range(5):
        n = train_data[l]
        l = np.array(encoder.encode(n), dtype=bool)
        
        active = layer.run_spatial_pooling(l, **spatial_pooling_params, learning_mode=False)
        predicted = layer.run_temporal_memory(active, **temporal_memory_params, 
                                            learning_mode=False, return_prediction=True)
        print(predicted)

  0%|          | 0/100 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 